# 방향
- 이중분류 할것이다: 사진이 주어졌을때 이게 개냐 ~ 아니냐 ~

# 할거
- 3개의 함수 : 학습, 검증 및 테스트, 예측
- 모델 : 층, 층별 노드, 퍼셉트론 수 조절한게 있어야 함

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import torch
import torch.nn.functional as F 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim 
import torchmetrics.functional as metrics

In [ ]:
cat_file = 'cat_df.csv'
dog_file = 'dog_df.csv'
cat_data = pd.read_csv(cat_file)
dog_data = pd.read_csv(dog_file)
data = pd.concat([cat_data, dog_data])

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
col_list = []
for i in range(1, 262146):
    col_list.append(f"pixel{i}")
data.columns = col_list
data.columns

In [ ]:
data.index

In [ ]:
type_list = []
for i in range(50):
    type_list.append('cat')
for i in range(50):
    type_list.append('dog')
# type_list

data['type'] = type_list

In [ ]:
data.columns

In [ ]:
data.shape

# 피처, 타겟 분리 & 정규화

In [ ]:
feature = data[data.columns[:-1]]
target = data[data.columns[-1]]

In [ ]:
print(f"feature.shape: {feature.shape}, target.shape: {target.shape}")

In [103]:
# cat을 0으로 dog를 1로 변경
target.replace('cat',0, inplace = True)
target.replace('dog',1, inplace = True)
scaled_target = target

In [104]:
# 피처를 정규화 시킴
scaled_feature = feature/512.

In [105]:
scaled_feature.shape

(100, 262145)

# 학습 데이터셋 준비 (학습용, 검증용, 테스트용)

In [106]:
# 사용자 정의 데이터셋 및 전체 데이터셋 준비

class all_dataset(Dataset):
    def __init__(self, feature_data, target_data):
        super().__init__()  # 어미를 상속받았다
        # 해당 클래스의 인스턴스 초기화함
        self.data = torch.FloatTensor(feature_data)
        self.target = torch.FloatTensor(target_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.target[index]

In [107]:
# 전체 DataSet 생성
feature_np = np.array(scaled_feature)
target_np = np.array(scaled_target)

animal_dataset = all_dataset(feature_np, target_np)

print(f"feature.shape: {animal_dataset.data.shape}, target.shape: {animal_dataset.target.shape}")

feature.shape: torch.Size([100, 262145]), target.shape: torch.Size([100])


In [108]:
# 학습용, 검증용, 테스트용 데이터셋 분리

In [109]:
seed_generator = torch.Generator().manual_seed(11)

train_size, valid_size, test_size = 0.7, 0.1, 0.2

train_dataset, valid_dataset, test_dataset = random_split(animal_dataset, [train_size, valid_size, test_size], generator = seed_generator)

In [110]:
print(f"train dataset length : {len(train_dataset)}")
print(f"valid dataset length : {len(valid_dataset)}")
print(f"test dataset length : {len(test_dataset)}")

train dataset length : 70
valid dataset length : 10
test dataset length : 20


# 데이터 로더 생성

In [111]:
batch = 5

train_loader = DataLoader(train_dataset, batch_size = batch)
val_loader = DataLoader(valid_dataset, batch_size = batch)
test_loader = DataLoader(test_dataset, batch_size = batch)

print(train_loader, val_loader, test_loader, sep='\n\n')

# 모델 준비

In [112]:
# 이중 분류  | 입력 피처 수 : 512 * 512  |  출력 피처 수 : 1 (0 or 1)

class myModel(nn.Module):
    def __init__(self, input_num, output_num):
        super().__init__()
        self.layer1 = nn.Linear(input_num, 100000)
        self.layer2 = nn.Linear(100000, 50000)
        self.layer3 = nn.Linear(50000, 25000)
        self.layer4 = nn.Linear(25000, 12000)
        self.layer5 = nn.Linear(12000, 6000)
        self.layer6 = nn.Linear(3000, 1000)
        self.layer7 = nn.Linear(1000, output_num)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.relu(x)
        x = self.layer4(x)
        x = self.relu(x)
        x = self.layer5(x)
        x = self.relu(x)
        x = self.layer6(x)
        x = self.relu(x)
        x = self.layer7(x)
        return x

### Q : 은닉층이면 relu 밖에 안됨? sigmoid,,, 흠,,, 왜 활성함수로 저거를 쓴 이유가 있어야할듯

In [119]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [118]:
# 학습 모델 인스턴스 생성
dogmodel = myModel(feature_np.shape[1], 2).to(device)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 104858000000 bytes.

In [115]:
dogmodel.parameters()

NameError: name 'dogmodel' is not defined

In [116]:
# 최적화 인스턴스 생성
optimizer = optim.SGD(dogmodel.parameters())

NameError: name 'dogmodel' is not defined

### Q. 왜 이 optimizer를 썻는가?

# 함수 정의 : training, testing

# 모델 저장

# 학습 진행 및 학습 결과 저장